# A simple REST service w/CRUD

We will go over a small example REST service to highlight a couple of python features

- Intro to pyproject.toml
- Demo of getting existing data, creating new data, and showing existing data
    - Show the code
    - Start the service in reload mode
    - Go to the swagger page
    - Run example
    - Run example with the httpx client

## The pyproject.toml file

Python has started strongly leaning into the TOML file world over yaml. A TOML file will unambiguously map to...well, a
map. The pyproject.toml has become standardized in [PEP-518](https://peps.python.org/pep-0518/) and is used by many
python tooling like poetry, autopep8, pyflake, hatch, pipenv, and many others.

We are using the poetry tool, and it has some important metadata extras.  

- It does the work that would normally be done in a requirements.txt file
    - the dependency resolver is more sophisticated than the default pip
- replaces `setuptools.py` to build a wheel and upload to PyPI

The pyproject.toml file's most import parts are:

- The version of the project
- The dependencies
    - Dependencies can declare a range, or exact specifications
    - Dependencies can be optional and installed with a `-E` extras flag
    - Dependencies can also be installed in groups like for development with `-G dev`
- Can specify alternative repositories (eg private repos)

In [ ]:
# Create a virtualenv and run 
!virtualenv -p python3.11.5 .venv
!poetry install --with dev

## Reminder on excursor

If you have the latest excursor, you can use it to install 

- asdf
- virtualenv
- poetry

It's been tested on fedora, amazonlinux and ubuntu.  Unfortunately, it's hard to test on a mac, because you can't truly
start with a clean system like you can with linux docker containers

```
python -m excursor.core.installer full
```

## Demo of ares: The code

Show off a little demo of the beginnings of a roleplaying game engine.  It can create and retrieve characters from a
local duckdb database.  duckdb is like sqlite on steroids.  Although it's meant for OLAP rather than OLTP apps, it has
full ACID guarantees, so it can be used for OLTP also.

- Show the initialization of the database
- Show the functions in service.py
- Show the model in db/characters/characters.py

In [ ]:
from pathlib import Path
import duckdb as dd
from duckdb import DuckDBPyConnection
from fastapi import FastAPI
from fastapi.responses import FileResponse
import uvicorn

from ares.models.character.character import Character, init_char_pq_path
import pyarrow as pa

# Initializing the database

We use duckdb to store and query the data.  duckdb is extremely fast (faster than Spark by a wide margin on data that 
can fit on a single node).  Perhaps most interestingly, it can query JSON data that's been stored in a newline delimited
format (eg NDJSON aka JSONL).

Here, if we don't already have a database, it will be created.  Note the `init_df` variable which _looks_ like it is not
being used.  But actually it is.  Duckdb will use the dataframe from the seeded initial parquet file (with the embedded
schema) and be able to use it like SQL Table.

> duckdb persistence
>
> By default, you do not have to give a name to connect, and it will use an in-memory database.  All data will be lost
> when the program stops though

If the db file does exist, then we load the `char_db` table.  A `testing` inner function will create a new random
character each time we start up the service.  This is an example of a python inner function, which is handy when you
want a _private_ function that is not accessible to outside code (technically, you can still get at it, but don't be a
bad programmer).

In [ ]:
# Paths for databases
ares_dir =Path(__file__).parent.parent

# Create or open the database our service will use
def init_db(db_name: str = "ares.db", test=True) -> DuckDBPyConnection:
    if not Path(db_name).exists():
        conn = dd.connect(db_name)
        init_df = conn.read_parquet(f"{init_char_pq_path}")
        conn.execute("CREATE TABLE char_db as SELECT * FROM init_df")
    else:
        conn = dd.connect(db_name)
        print(conn.sql("SHOW ALL TABLES"))
        print(conn.sql("SELECT * FROM char_db"))


    def new_record():
        example = Character.random_character()
        schema = Character.arrow_schema()
        tbl = pa.Table.from_pylist([example.model_dump()], schema=schema)
        sql_cmd = f"INSERT INTO char_db SELECT * FROM tbl RETURNING *"
        print(sql_cmd)
        df = conn.sql(sql_cmd)
        print(df)

    if test:
        new_record()
    return conn

## The actual REST methods

This is how the actual methods are implemented.  I won't get into much detail other than this:

- the route and http method are defined by the app.method and the route path
- the parameters to a method are either query params, path params, or request body
- get_parquet: saves the DB as parquet file and downloads to user
- post_character: creates a new character and inserts into the db
- get_character: gets a character by uuid
- get_characters: returns a list of all characters

In [ ]:
# Create the FastAPI application
app = FastAPI()

# /v1/characters
char_ept = "/v1/characters/"

# Create the database connection
conn = init_db()

@app.get(f"{char_ept}parquet")
async def get_parquet():
    df = conn.sql("SELECT * FROM char_db")
    pq_path = Path("parquet/saved.parquet")
    if pq_path.exists():
        pq_path.unlink()
    pq_path.parent.mkdir(parents=True, exist_ok=True)
    df.write_parquet("parquet/saved.parquet", compression="snappy")
    return FileResponse(pq_path)


@app.post(char_ept)
async def post_character(char: Character):
    model = char.model_dump()
    print(model)
    schema = Character.arrow_schema()
    tbl = pa.Table.from_pylist([model], schema=schema)
    sql_cmd = f"INSERT INTO char_db SELECT * FROM tbl RETURNING *"
    print(sql_cmd)
    df = conn.sql(sql_cmd)
    print(df)


@app.get(f"{char_ept}/{{uid}}")
async def get_character(uid: str):
    sql_cmd = f"SELECT * FROM char_db WHERE uid = '{uid}'"
    df = conn.sql(sql_cmd)
    batch = df.to_arrow_table()
    data = batch.to_pylist()[0]
    return data


@app.get(char_ept)
async def get_characters():
    df = conn.sql("SELECT * FROM char_db")
    batch = df.to_arrow_table()
    data = batch.to_pylist()
    chars: list[Character] = []
    for d in data:
        d["skills"] = {name: lvl for name, lvl in d["skills"]}
        chars.append(Character.model_validate(d))
    return chars


## The Character model

FastAPI makes heavy use of pydantic for data types that can be passed as request bodies, or returned as json responses. 
The important parts here are:

- Helper functions `die` and `best_of`
- Character which inherits from pydantic's `BaseModel`
    - the fields must all be typed
    - staticmethod to generate a random character
    - arrow_schema which is used for duckdb and arrow/parquet

In [ ]:
from pathlib import Path
from random import randint, random
from uuid import uuid4

import pyarrow as pa
from pydantic import BaseModel
init_char_pq_path = Path(__file__).parent.parent.parent / "db/characters/character_init.parquet"

def die(num: int, size: int = 20):
    def pool():
        for _ in range(num):
            yield randint(1, size)
    return pool

def best_of(pool: list[int], amount: int):
    pool_size = len(pool)
    if amount > pool_size:
        amount = pool_size
    start_from = pool_size - amount
    return sorted(pool)[start_from:]

class Character(BaseModel):
    uid: str
    player: str
    player_id: str
    force: int
    speed: int
    kinesthesia: int
    wit: int
    insight: int
    discipline: int
    height: float
    weight: float
    skills: dict[str, int]

    @staticmethod
    def random_character() -> "Character":
        pool = die(4, 6)
        return Character(
            uid=str(uuid4()),
            player="Sean Toner",
            player_id=str(uuid4()),
            force=sum(best_of(list(pool()), 3)),
            speed=sum(best_of(list(pool()), 3)),
            kinesthesia=sum(best_of(list(pool()), 3)),
            wit=sum(best_of(list(pool()), 3)),
            insight=sum(best_of(list(pool()), 3)),
            discipline=sum(best_of(list(pool()), 3)),
            height=50.0 + random() * 50,
            weight=60 + random() * 40,
            skills={
                "1HSword": sum(best_of(list(pool()), 3)),
                "Climbing": sum(best_of(list(pool()), 3))
            }
        )

    @staticmethod
    def arrow_schema():
        return pa.schema([
            ("uid", pa.string()),
            ('player', pa.string()),
            ("player_id", pa.string()),
            ("force", pa.int64()),
            ("speed", pa.int64()),
            ("kinesthesia", pa.int64()),
            ("wit", pa.int64()),
            ("insight", pa.int64()),
            ("discipline", pa.int64()),
            ("height", pa.float64()),
            ("weight", pa.float64()),
            ("skills", pa.map_(pa.string(), pa.int64()))
        ])

In [ ]:
import httpx
from ares.models.character.character import Character


def test_add():
    example = Character.random_character()
    char = Character.model_validate(example)
    resp = httpx.post("http://127.0.0.1:8000/v1/characters/", json=char.model_dump())
    print(resp)
    print(resp.content)

def test_list():
    resp = httpx.get("http://127.0.0.1:8000/v1/characters/")
    print(resp.json())

def test_by_id(uuid: str):
    resp = httpx.get(f"http://127.0.0.1:8000/v1/characters/{uuid}")
    print(resp)
    print(resp.json())

In [ ]:
test_add()

In [ ]:
test_list()

In [ ]:
test_by_id("9ac41b52-5d63-4897-b168-0efc4868eb51")